In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
%cd /content/drive/MyDrive/LumiNet_Files/lumiply-colab/

/content/drive/MyDrive/LumiNet_Files/lumiply-colab


In [2]:
# LumiNet 패키지 설치
!pip install -r requirements.txt
# Flask 패키지 설치
!pip install flask flask-cors requests pillow pyngrok torchvision flask_cors

In [3]:
from huggingface_hub import login

login()

In [4]:
!lsof -i | grep :5000 # 아무것도 안 떠야함

In [ ]:
# !kill -9 712

In [5]:
# =========================
# [Cell 1] 모델/엔진 초기화 (약 5~10분 소요)
# =========================

import sys
import os
import torch
import cv2
import numpy as np
from PIL import Image
from omegaconf import OmegaConf

from ldm.util import instantiate_from_config
from cldm.model import load_state_dict
from cldm.ddim_hacked import DDIMSampler
import einops

# 경로 설정 (Colab 경로에 맞게 수정 필요)
# BASE_MODEL_PATH = "./crossattn_checkpoints/best_crossattn_offwhite.ckpt"
BASE_MODEL_PATH = "./ckpt/trained_crossattn.ckpt"
CONFIG_PATH = "./models/cldm_v21_LumiNet.yaml"

# 전역 변수 (모델을 메모리에 계속 유지하기 위함)
global_model = None
global_sampler = None
current_adapter_color = None

def initialize_engine():
    """베이스 모델을 메모리에 로드합니다."""
    global global_model, global_sampler

    if global_model is not None:
        print("모델이 이미 로드되어 있습니다.")
        return

    print("베이스 모델 로딩 시작... (최초 1회)")
    config = OmegaConf.load(CONFIG_PATH)
    model = instantiate_from_config(config.model).cpu()
    model.add_new_layers()

    if os.path.exists(BASE_MODEL_PATH):
        model.load_state_dict(load_state_dict(BASE_MODEL_PATH, location='cpu'), strict=False)
        print(f"Base Model 로드 완료: {BASE_MODEL_PATH}")
    else:
        raise FileNotFoundError(f"Base Model 없음: {BASE_MODEL_PATH}")

    new_decoder = True #
    if new_decoder: #
        ae_checkpoint = "./ckpt/new_decoder.ckpt" #
        model.change_first_stage(ae_checkpoint) #

    model.cuda()
    model.eval()

    global_model = model
    global_sampler = DDIMSampler(model)
    print("엔진 준비 완료!")

def switch_adapter(target_color):
    """요청된 색상으로 어댑터 가중치만 교체합니다."""
    global global_model, current_adapter_color

    if current_adapter_color == target_color:
        print(f"이미 {target_color} 어댑터가 적용되어 있습니다.")
        return

    weights_path = f"./adaptors/adaptor_{target_color}.pth"

    if not os.path.exists(weights_path):
        print(f"(White 모드로 진행)")
        current_adapter_color = "none"
        return

    print(f"어댑터 교체 중... ({current_adapter_color} -> {target_color})")
    checkpoint = torch.load(weights_path, map_location='cpu')

    # Hot-Swapping (모델을 끄지 않고 가중치만 덮어쓰기)
    if 'light_encoder' in checkpoint:
        global_model.control_model.prior_extracter.model_latents.light_encoder.load_state_dict(checkpoint['light_encoder'])
    if 'light_decoder' in checkpoint:
        global_model.control_model.prior_extracter.light_decoder.load_state_dict(checkpoint['light_decoder'])

    # ==================================================
    # 정밀 검증 (Verification)
    print("\n🔍 가중치 적용 상태 검증 중...")
    is_valid = True

    # Encoder 검증
    if 'light_encoder' in checkpoint:
        key = list(checkpoint['light_encoder'].keys())[0]

        # [수정] 비교 전에 둘 다 CPU로 이동
        file_val = checkpoint['light_encoder'][key].cpu()
        model_val = global_model.control_model.prior_extracter.model_latents.light_encoder.state_dict()[key].cpu()

        if torch.allclose(file_val, model_val, atol=1e-6):
            print(f"Encoder 검증 통과! (Key: {key})")
        else:
            print(f"Encoder 검증 실패! 값이 다릅니다.")
            is_valid = False

    # Adaptor 검증
    if 'light_decoder' in checkpoint:
        key = list(checkpoint['light_decoder'].keys())[0]

        # [수정] 비교 전에 둘 다 CPU로 이동
        file_val = checkpoint['light_decoder'][key].cpu()
        model_val = global_model.control_model.prior_extracter.light_decoder.state_dict()[key].cpu()

        if torch.allclose(file_val, model_val, atol=1e-6):
            print(f"Adaptor 검증 통과! (Key: {key})")
        else:
            print(f"Adaptor 검증 실패! 값이 다릅니다.")
            is_valid = False

    if not is_valid:
        print("경고: 가중치가 제대로 적용되지 않았을 수 있습니다. 코드를 확인하세요.")
    else:
        print("모든 가중치가 완벽하게 적용되었습니다. (Color Mode ON)")
    # ==================================================

    current_adapter_color = target_color
    print(f"{target_color} 어댑터 장착 완료")

def run_inference_single_image(off_path, color, guidance_scale=9.0, ddim_steps=50):
    """
    off_path: 합성된 'off' 이미지 경로 (원본 해상도)
    color: 'red', 'orange', 'yellow', 'green', 'blue', 'purple' 등
    반환: 저장된 결과 이미지 경로
    """
    switch_adapter(color)
    new_decoder = True #

    print(f"\nSingle Image 처리 시작: {off_path} (Color: {color})")

    img_off = Image.open(off_path).convert("RGB")
    orig_w, orig_h = img_off.size
    img_off_resized = img_off.resize((512, 512), Image.BICUBIC)
    img_ref_resized = Image.new("RGB", (512, 512), (255, 255, 255))

    t_off = torch.from_numpy(np.array(img_off_resized).astype(np.float32) / 255.0).permute(2,0,1).unsqueeze(0).cuda()
    t_ref = torch.from_numpy(np.array(img_ref_resized).astype(np.float32) / 255.0).permute(2,0,1).unsqueeze(0).cuda()

    hint = torch.cat((t_off, t_ref), dim=1)

    input_image_full = cv2.imread(off_path)[..., ::-1] / 255.0 #
    inp = cv2.resize(input_image_full, (512, 512)) #
    input_img = torch.from_numpy(inp.copy()).float().cuda() #
    input_img = input_img.unsqueeze(0) #
    input_img = einops.rearrange(input_img, 'b h w c -> b c h w').clone() #

    # 추론
    with torch.no_grad():
        c_cat = hint
        c = global_model.get_unconditional_conditioning(c_cat.shape[0])
        cond = {"c_concat": [c_cat], "c_crossattn": [c]}
        shape = (4, 512 // 8, 512 // 8)

        samples, _ = global_sampler.sample(
            ddim_steps, 1, shape, cond, verbose=False,
            unconditional_guidance_scale=guidance_scale
        )

        # x_sample = global_model.decode_first_stage(samples)
        # x_sample = torch.clamp((x_sample + 1.0) / 2.0, min=0.0, max=1.0)
        if new_decoder: #
            ae_hs = global_model.encode_first_stage(input_img * 2 - 1)[1] #
            x_sample = global_model.decode_new_first_stage(samples, ae_hs) #
        else: #
            x_sample = global_model.decode_first_stage(samples) #
        x_sample = torch.clamp((x_sample + 1.0) / 2.0, min=0.0, max=1.0) #

        result_numpy = x_sample.cpu().permute(0, 2, 3, 1).numpy()[0] * 255
        result_numpy = result_numpy.astype(np.uint8)
        result_final = cv2.resize(result_numpy, (orig_w, orig_h), interpolation=cv2.INTER_LANCZOS4)

        # 저장 경로는 off_path 기준으로 output_{color}.jpg
        root, fname = os.path.split(off_path)
        save_path = os.path.join(root, f"output_{color}.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(result_final, cv2.COLOR_RGB2BGR))
        print(f"Saved: {save_path}")
    return save_path

# 기존 run_process(inference_root, color)는 여러 파일 batch 용으로 남겨두고,
# 온라인 요청용으로는 run_inference_single_image(...)를 핵심으로 사용.

initialize_engine()

베이스 모델 로딩 시작... (최초 1회)


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/core/lightning.py:2058: DeprecationWarning: `torch.distributed._sharded_tensor` will be deprecated, use `torch.distributed._shard.sharded_tensor` instead
  from torch.distributed._sharded_tensor import pre_load_state_dict_hook, state_dict_hook


ControlLDM: Running in v-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, c

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 1024 and using 20 heads.
Se

In [6]:
# =========================
# [Cell 2] Flask + ngrok 서버
# =========================

from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
from PIL import Image
import io
import os
from datetime import datetime

app = Flask(__name__)
CORS(app)

INFERENCE_ROOT = "./images/inference"
os.makedirs(INFERENCE_ROOT, exist_ok=True)

PUBLIC_URL = None  # [Cell 3] 에서 ngrok.connect 후 채움

@app.before_request
def handle_ngrok_warning():
    """ngrok 브라우저 경고 페이지 우회"""
    if request.headers.get("ngrok-skip-browser-warning"):
        pass

@app.before_request
def log_request_info():
    print("=" * 50)
    print(f"요청 수신: {request.method} {request.path}")
    print(f"Headers: {dict(request.headers)}")
    if request.method == "POST":
        print(f"Form data keys: {list(request.form.keys())}")
        print(f"Files keys: {list(request.files.keys())}")
    print("=" * 50)

@app.route("/health", methods=["GET"])
def health_check():
    print("Health check 요청 수신")
    return jsonify(
        {
            "status": "healthy",
            "message": "Colab 서버가 실행 중입니다.",
            "timestamp": datetime.now().isoformat(),
        }
    ), 200

@app.route("/process", methods=["POST"])
def process_image():
    """
    FastAPI에서 전송된 합성 이미지를 받아,
    /images/inference/{job_id}/off.png 로 저장한 뒤,
    요청된 단일 색상(color)에 대해서만 output_{color}.jpg 를 생성하고,
    해당 색상 결과 + 입력(off) 이미지 URL 을 FastAPI/프론트에 반환.
    """
    try:
        print("=" * 50)
        print("Process 엔드포인트 호출됨!")
        print("=" * 50)

        if "image" not in request.files:
            print("이미지 파일이 없습니다.")
            return jsonify({"error": "이미지 파일이 없습니다."}), 400

        file = request.files["image"]
        job_id = request.form.get("job_id")
        callback_url = request.form.get("callback_url")
        color_param = request.form.get("color")  # ★ 추가: 단일 색상 지정

        print(f"📦 수신된 데이터:")
        print(f"   - job_id: {job_id}")
        print(f"   - callback_url: {callback_url}")
        print(f"   - color: {color_param}")
        print(f"   - 파일명: {file.filename}")
        print(f"   - 파일 타입: {file.content_type}")

        if not job_id:
            print("job_id가 없습니다.")
            return jsonify({"error": "job_id가 없습니다."}), 400

        # 지원 색상 검증 (단일 색상만 허용)
        base_colors = ["white", "red", "orange", "yellow", "green", "blue", "purple"]
        if not color_param:
            return jsonify({"error": "color 파라미터가 필요합니다."}), 400
        if color_param not in base_colors:
            return jsonify({"error": f"지원하지 않는 색상입니다: {color_param}"}), 400

        color_key = color_param

        if not callback_url:
            print("callback_url이 없습니다. 콜백을 전송하지 않습니다.")

        print(f"[{job_id}] 이미지 처리 시작")

        # 1) job 전용 디렉토리 생성
        job_dir = os.path.join(INFERENCE_ROOT, job_id)
        os.makedirs(job_dir, exist_ok=True)

        # 2) 입력 이미지를 off.png 로 저장
        off_path = os.path.join(job_dir, "off.png")
        image_bytes = file.read()
        with open(off_path, "wb") as f:
            f.write(image_bytes)

        print(f"[{job_id}] 입력 이미지 저장 완료: {off_path}")
        print(f"[{job_id}] 이미지 크기: {len(image_bytes)} bytes")

        from pathlib import Path

        images = {}
        # for ck in color_key:
        print(f"[{job_id}] === {color_key} 색상 처리 시작 ===")
        output_path = run_inference_single_image(off_path=off_path, color=color_key)

        if os.path.exists(output_path):
            rel_path = f"/static/inference/{job_id}/{Path(output_path).name}"
            images[color_key] = f"{PUBLIC_URL}{rel_path}"
            print(f"[{job_id}] {color_key} 결과 URL: {images[color_key]}")
        else:
            print(f"[{job_id}] {color_key} 결과 이미지가 존재하지 않습니다: {output_path}")

        # 인풋(off) 이미지 URL
        input_rel = f"/static/inference/{job_id}/off.png"
        input_image_url = f"{PUBLIC_URL}{input_rel}"

        response_data = {
            "job_id": job_id,
            "status": "completed",
            "result": {
                "images": images,
                "input_image_url": input_image_url,
            },
            "message": "색상 이미지 생성 완료",
        }

        # 콜백
        if callback_url:
            try:
                print(f"[{job_id}] FastAPI로 결과 전송 중...")
                callback_response = requests.post(
                    callback_url,
                    json=response_data,
                    timeout=60,
                    headers={"ngrok-skip-browser-warning": "true"},
                )
                print(f"[{job_id}] 콜백 응답 상태: {callback_response.status_code}")
            except Exception as e:
                print(f"[{job_id}] 콜백 전송 오류: {str(e)}")

        print(f"[{job_id}] 처리 완료")
        return jsonify(response_data), 200

    except Exception as e:
        error_msg = str(e)
        print(f"처리 오류: {error_msg}")
        import traceback
        traceback.print_exc()
        return jsonify({"success": False, "error": error_msg}), 500

In [7]:
# =========================
# [Cell 3] ngrok + Flask 서버 (백그라운드) 설정
# =========================

from pyngrok import ngrok
from werkzeug.middleware.shared_data import SharedDataMiddleware
from werkzeug.serving import make_server
import threading
import time

# 1) 정적 파일 서빙 설정 (Flask app은 이미 Cell 2에서 정의되어 있다고 가정)
app.wsgi_app = SharedDataMiddleware(app.wsgi_app, {
    "/static/inference": INFERENCE_ROOT,
})

# 2) Flask 서버를 백그라운드 스레드에서 돌리기 위한 클래스
class FlaskServerThread(threading.Thread):
    def __init__(self, app, host="0.0.0.0", port=5000):
        super().__init__()
        self.app = app
        self.host = host
        self.port = port
        self.server = None
        self.ctx = None
        self.daemon = True  # 노트북 종료 시 같이 죽도록

    def run(self):
        print(f"Flask 서버 시작... ({self.host}:{self.port})")
        # WSGI 서버 인스턴스를 직접 잡아둔다
        self.server = make_server(self.host, self.port, self.app)
        self.ctx = self.app.app_context()
        self.ctx.push()
        try:
            self.server.serve_forever()
        finally:
            self.server.server_close()
            self.ctx.pop()
            print("Flask 서버 종료 완료")

    def shutdown(self):
        if self.server:
            print("Flask 서버 shutdown 요청")
            self.server.shutdown()  # serve_forever 루프 깨기

# 3) 이미 열려 있는 ngrok 터널이 있으면 정리
if "NGROK_TUNNEL" in globals() and NGROK_TUNNEL is not None:
    try:
        print("기존 ngrok 터널 닫는 중:", NGROK_TUNNEL.public_url)
        NGROK_TUNNEL.close()
    except Exception as e:
        print("기존 터널 close 중 오류:", e)
    NGROK_TUNNEL = None

# 4) Flask 서버 스레드 시작 (백그라운드)
flask_server = FlaskServerThread(app, host="0.0.0.0", port=5000)
flask_server.start()

# 서버가 뜰 시간 약간 대기
time.sleep(2)
print("Flask 서버 스레드 실행 중")

# 5) ngrok 설정
NGROK_TOKEN = "NGROK_TOKEN_입력"
ngrok.set_auth_token(NGROK_TOKEN)
ngrok.kill()  # 혹시 남아 있을 수 있는 ngrok 프로세스 정리

# 새 터널 생성 (Flask가 5000 포트에서 이미 LISTEN 중이어야 함)
NGROK_TUNNEL = ngrok.connect(5000)

public_url = getattr(NGROK_TUNNEL, "public_url", str(NGROK_TUNNEL))

if public_url.startswith("http://"):
    public_url = public_url.replace("http://", "https://")
elif not public_url.startswith("https://"):
    public_url = f"https://{public_url}"

PUBLIC_URL = public_url
process_url = f"{public_url}/process"

print("=" * 50)
print("✅ Colab 서버가 시작되었습니다!")
print(f"🌐 Public URL: {public_url}")
print("📝 FastAPI의 COLAB_WEBHOOK_URL 환경 변수에 다음 URL을 설정하세요:")
print(f"   {process_url}")
print("=" * 50)
print("\n💡 복사할 URL:")
print(f"   {process_url}")
print("=" * 50)


Flask 서버 시작... (0.0.0.0:5000)
Flask 서버 스레드 실행 중
✅ Colab 서버가 시작되었습니다!
🌐 Public URL: https://undented-preinflectional-madeline.ngrok-free.dev
📝 FastAPI의 COLAB_WEBHOOK_URL 환경 변수에 다음 URL을 설정하세요:
   https://undented-preinflectional-madeline.ngrok-free.dev/process

💡 복사할 URL:
   https://undented-preinflectional-madeline.ngrok-free.dev/process


In [8]:
# # Flask 서버 종료

# flask_server.shutdown()

Flask 서버 shutdown 요청
Flask 서버 종료 완료


## 테스트 코드

In [ ]:
# !pwd

/content/drive/MyDrive/LumiNet_Files/LumiNet-lee


In [ ]:
# import requests
# from PIL import Image
# import io

# colors = ["white", "red", "orange", "yellow", "green", "blue", "purple"]
# job_id = "debug-job"

# for color in colors:
#     print("=" * 50)
#     print(f"Sending request for color: {color}")

#     with open("./23_off.jpeg", "rb") as f:
#         resp = requests.post(
#             f"{PUBLIC_URL}/process",
#             # files={"image": ("off.png", f, "image/png")},
#             files={"image": f},
#             data={"job_id": job_id, "callback_url": "", "color": color},
#             headers={"ngrok-skip-browser-warning": "true"},
#             timeout=300,
#         )

#     print("Status:", resp.status_code)
#     print("Response:", resp.text[:400])  # 너무 길어지지 않게 앞부분만 출력

Sending request for color: white
요청 수신: POST /process
Headers: {'Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'User-Agent': 'python-requests/2.32.4', 'Content-Length': '717510', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Type': 'multipart/form-data; boundary=ca8e0f2a1c067cae162ce6be65f4e061', 'Ngrok-Skip-Browser-Warning': 'true', 'X-Forwarded-For': '34.126.159.241', 'X-Forwarded-Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'X-Forwarded-Proto': 'https'}
Form data keys: ['job_id', 'callback_url', 'color']
Files keys: ['image']
Process 엔드포인트 호출됨!
📦 수신된 데이터:
   - job_id: debug-job
   - callback_url: 
   - color: white
   - 파일명: 23_off.jpeg
   - 파일 타입: None
callback_url이 없습니다. 콜백을 전송하지 않습니다.
[debug-job] 이미지 처리 시작
[debug-job] 입력 이미지 저장 완료: ./images/inference/debug-job/off.png
[debug-job] 이미지 크기: 717082 bytes
[debug-job] === white 색상 처리 시작 ===
(White 모드로 진행)

Single Image 처리 시작: ./images/inference/debug-job/off.png (Color: white)
Data s

DDIM Sampler: 100%|██████████| 50/50 [00:06<00:00,  7.65it/s]
INFO:werkzeug:127.0.0.1 - - [08/Dec/2025 15:31:40] "POST /process HTTP/1.1" 200 -


Saved: ./images/inference/debug-job/output_white.jpg
[debug-job] white 결과 URL: https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_white.jpg
[debug-job] 처리 완료
Status: 200
Response: {"job_id":"debug-job","message":"\uc0c9\uc0c1 \uc774\ubbf8\uc9c0 \uc0dd\uc131 \uc644\ub8cc","result":{"images":{"white":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_white.jpg"},"input_image_url":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/off.png"},"status":"completed"}

Sending request for color: red
요청 수신: POST /process
Headers: {'Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'User-Agent': 'python-requests/2.32.4', 'Content-Length': '717508', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Type': 'multipart/form-data; boundary=26c2959e32540c703eadf1d8e73488d1', 'Ngrok-Skip-Browser-Warning': 'true', 'X-Forwarded-For': '34.126.159.241', 'X-Forward

DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.96it/s]
INFO:werkzeug:127.0.0.1 - - [08/Dec/2025 15:31:59] "POST /process HTTP/1.1" 200 -


Saved: ./images/inference/debug-job/output_red.jpg
[debug-job] red 결과 URL: https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_red.jpg
[debug-job] 처리 완료
Status: 200
Response: {"job_id":"debug-job","message":"\uc0c9\uc0c1 \uc774\ubbf8\uc9c0 \uc0dd\uc131 \uc644\ub8cc","result":{"images":{"red":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_red.jpg"},"input_image_url":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/off.png"},"status":"completed"}

Sending request for color: orange
요청 수신: POST /process
Headers: {'Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'User-Agent': 'python-requests/2.32.4', 'Content-Length': '717511', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Type': 'multipart/form-data; boundary=834074b8b21b54950d4e6d92a2137a9c', 'Ngrok-Skip-Browser-Warning': 'true', 'X-Forwarded-For': '34.126.159.241', 'X-Forwarded-Host

...

Saved: ./images/inference/debug-job/output_green.jpg
[debug-job] green 결과 URL: https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_green.jpg
[debug-job] 처리 완료
Status: 200
Response: {"job_id":"debug-job","message":"\uc0c9\uc0c1 \uc774\ubbf8\uc9c0 \uc0dd\uc131 \uc644\ub8cc","result":{"images":{"green":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_green.jpg"},"input_image_url":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/off.png"},"status":"completed"}

Sending request for color: blue
요청 수신: POST /process
Headers: {'Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'User-Agent': 'python-requests/2.32.4', 'Content-Length': '717509', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Type': 'multipart/form-data; boundary=17fbda1a60db98ba7f35a9701158e34f', 'Ngrok-Skip-Browser-Warning': 'true', 'X-Forwarded-For': '34.126.159.241', 'X-Forwar

DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.93it/s]
INFO:werkzeug:127.0.0.1 - - [08/Dec/2025 15:33:23] "POST /process HTTP/1.1" 200 -


Saved: ./images/inference/debug-job/output_blue.jpg
[debug-job] blue 결과 URL: https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_blue.jpg
[debug-job] 처리 완료
Status: 200
Response: {"job_id":"debug-job","message":"\uc0c9\uc0c1 \uc774\ubbf8\uc9c0 \uc0dd\uc131 \uc644\ub8cc","result":{"images":{"blue":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_blue.jpg"},"input_image_url":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/off.png"},"status":"completed"}

Sending request for color: purple
요청 수신: POST /process
Headers: {'Host': 'undented-preinflectional-madeline.ngrok-free.dev', 'User-Agent': 'python-requests/2.32.4', 'Content-Length': '717511', 'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br, zstd', 'Content-Type': 'multipart/form-data; boundary=70370bd63a99bc476bd25a34e94218a8', 'Ngrok-Skip-Browser-Warning': 'true', 'X-Forwarded-For': '34.126.159.241', 'X-Forwarded

DDIM Sampler: 100%|██████████| 50/50 [00:05<00:00,  8.86it/s]
INFO:werkzeug:127.0.0.1 - - [08/Dec/2025 15:33:47] "POST /process HTTP/1.1" 200 -


Saved: ./images/inference/debug-job/output_purple.jpg
[debug-job] purple 결과 URL: https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_purple.jpg
[debug-job] 처리 완료
Status: 200
Response: {"job_id":"debug-job","message":"\uc0c9\uc0c1 \uc774\ubbf8\uc9c0 \uc0dd\uc131 \uc644\ub8cc","result":{"images":{"purple":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/output_purple.jpg"},"input_image_url":"https://undented-preinflectional-madeline.ngrok-free.dev/static/inference/debug-job/off.png"},"status":"completed"}

